# Model 2
### 2 layer LSTM

2018-11-14

uses Nvidia Tesla P100 GPU  
uses .to(device)

***

### Hyperparameters

1. seqeuence_len / n_word = 512 (previous: 128)
2. batch_size / n_seq = 128
3. hidden_dim = 512
4. top_k words = 3 (previous: 5)
5. predict seq_len = 512 (previous: 1024)
6. epoch = 300


1. log every 100 batches
2. save model every 20 epochs

***

In [1]:
import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

import keras
from keras.utils import to_categorical

import time

Using TensorFlow backend.


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(device)

cuda:0


In [3]:
# replace with any text file containing full set of data
music_data = 'txt-files/notewise/beethoven.txt'

with open(music_data, 'r') as file:
    text = file.read()

In [4]:
# get vocabulary set
words = sorted(tuple(set(text.split())))
n = len(words)

# create word-integer encoder/decoder
word2int = dict(zip(words, list(range(n))))
int2word = dict(zip(list(range(n)), words))

# encode all words in dataset into integers
encoded = np.array([word2int[word] for word in text.split()])

In [5]:
# define model using the pytorch nn module
class WordLSTM(nn.ModuleList):
    
    def __init__(self, sequence_len, vocab_size, hidden_dim, batch_size):
        super(WordLSTM, self).__init__()
        
        # init the hyperparameters
        self.vocab_size = vocab_size
        self.sequence_len = sequence_len
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        
        # first layer lstm cell
        self.lstm_1 = nn.LSTMCell(input_size=vocab_size, hidden_size=hidden_dim)
        
        # second layer lstm cell
        self.lstm_2 = nn.LSTMCell(input_size=hidden_dim, hidden_size=hidden_dim)
        
        # dropout layer
        self.dropout = nn.Dropout(p=0.5)
        
        # fully connected layer
        self.fc = nn.Linear(in_features=hidden_dim, out_features=vocab_size)
        
    # forward pass in training   
    def forward(self, x, hc):
        """
            accepts 2 arguments: 
            1. x: input of each batch 
                - shape 128*149 (batch_size*vocab_size)
            2. hc: tuple of init hidden, cell states 
                - each of shape 128*512 (batch_size*hidden_dim)
        """
        
        # create empty output seq
        output_seq = torch.empty((self.sequence_len,
                                  self.batch_size,
                                  self.vocab_size))
        # if using gpu        
        output_seq = output_seq.to(device)
        
        # init hidden, cell states for lstm layers
        hc_1, hc_2 = hc, hc
        
        # for t-th word in every sequence 
        for t in range(self.sequence_len):
            
            # layer 1 lstm
            hc_1 = self.lstm_1(x[t], hc_1)
            h_1, c_1 = hc_1
            
            # layer 2 lstm
            hc_2 = self.lstm_2(h_1, hc_2)
            h_2, c_2 = hc_2
            
            # dropout and fully connected layer
            output_seq[t] = self.fc(self.dropout(h_2))
            
        return output_seq.view((self.sequence_len * self.batch_size, -1))
          
    def init_hidden(self):
        
        # initialize hidden, cell states for training
        # if using gpu
        return (torch.zeros(self.batch_size, self.hidden_dim).to(device),
                torch.zeros(self.batch_size, self.hidden_dim).to(device))
    
    def init_hidden_generator(self):
        
        # initialize hidden, cell states for prediction of 1 sequence
        # if using gpu
        return (torch.zeros(1, self.hidden_dim).to(device),
                torch.zeros(1, self.hidden_dim).to(device))
    
    def predict(self, seed_seq, top_k=5, pred_len=128):
        """
            accepts 3 arguments: 
            1. seed_seq: seed string sequence for prediction (prompt)
            2. top_k: top k words to sample prediction from
            3. pred_len: number of words to generate after the seed seq
        """
        
        # set evaluation mode
        self.eval()
        
        # split string into list of words
        seed_seq = seed_seq.split()
        
        # get seed sequence length
        seed_len = len(seed_seq)
        
        # create output sequence
        out_seq = np.empty(seed_len+pred_len)
        
        # append input seq to output seq
        out_seq[:seed_len] = np.array([word2int[word] for word in seed_seq])
 
        # init hidden, cell states for generation
        hc = self.init_hidden_generator()
        hc_1, hc_2 = hc, hc
        
        # feed seed string into lstm
        # get the hidden state set up
        for word in seed_seq[:-1]:
            
            # encode starting word to one-hot encoding
            word = to_categorical(word2int[word], num_classes=self.vocab_size)

            # add batch dimension
            word = torch.from_numpy(word).unsqueeze(0)
            # if using gpu
            word = word.to(device) 
            
            # layer 1 lstm
            hc_1 = self.lstm_1(word, hc_1)
            h_1, c_1 = hc_1
            
            # layer 2 lstm
            hc_2 = self.lstm_2(h_1, hc_2)
            h_2, c_2 = hc_2
        
        word = seed_seq[-1]
        
        # encode starting word to one-hot encoding
        word = to_categorical(word2int[word], num_classes=self.vocab_size)

        # add batch dimension
        word = torch.from_numpy(word).unsqueeze(0)
        # if using gpu
        word = word.to(device) 

        # forward pass
        for t in range(pred_len):
            
            # layer 1 lstm
            hc_1 = self.lstm_1(word, hc_1)
            h_1, c_1 = hc_1
            
            # layer 2 lstm
            hc_2 = self.lstm_2(h_1, hc_2)
            h_2, c_2 = hc_2
            
            # fully connected layer without dropout (no need)
            output = self.fc(h_2)
            
            # software to get probabilities of output options
            output = F.softmax(output, dim=1)
            
            # get top k words and corresponding probabilities
            p, top_word = output.topk(top_k)
            # if using gpu           
            p = p.cpu()
            
            # sample from top k words to get next word
            p = p.detach().squeeze().numpy()
            top_word = torch.squeeze(top_word)
            
            word = np.random.choice(top_word, p = p/p.sum())
            
            # add word to sequence
            out_seq[seed_len+t] = word
            
            # encode predicted word to one-hot encoding for next step
            word = to_categorical(word, num_classes=self.vocab_size)
            word = torch.from_numpy(word).unsqueeze(0)
            # if using gpu
            word = word.to(device)
            
        return out_seq

In [6]:
def get_batches(arr, n_seqs, n_words):
    """
        create generator object that returns batches of input (x) and target (y).
        x of each batch has shape 128*128*149 (batch_size*seq_len*vocab_size).
        
        accepts 3 arguments:
        1. arr: array of words from text data
        2. n_seq: number of sequence in each batch (aka batch_size)
        3. n_word: number of words in each sequence
    """
    
    # compute total elements / dimension of each batch
    batch_total = n_seqs * n_words
    
    # compute total number of complete batches
    n_batches = arr.size//batch_total
    
    # chop array at the last full batch
    arr = arr[: n_batches* batch_total]
    
    # reshape array to matrix with rows = no. of seq in one batch
    arr = arr.reshape((n_seqs, -1))
    
    # for each n_words in every row of the dataset
    for n in range(0, arr.shape[1], n_words):
        
        # chop it vertically, to get the input sequences
        x = arr[:, n:n+n_words]
        
        # init y - target with shape same as x
        y = np.zeros_like(x)
        
        # targets obtained by shifting by one
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], x[:, n+n_words]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        
        # yield function is like return, but creates a generator object
        yield x, y   

In [7]:
# compile the network - sequence_len, vocab_size, hidden_dim, batch_size
net = WordLSTM(sequence_len=512, vocab_size=len(word2int), hidden_dim=512, batch_size=128)
# if using gpu
net.to(device)

# define the loss and the optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# split dataset into 90% train and 10% using index
val_idx = int(len(encoded) * (1 - 0.1))
train_data, val_data = encoded[:val_idx], encoded[val_idx:]

# empty list for the validation losses
val_losses = list()

# empty list for the samples
samples = list()

In [ ]:
for i, (x,y) in enumerate(get_batches(train_data, 128, 512)):
    print(i,(x,y))

In [ ]:
for i in net.parameters():
    print(i)

In [8]:
# track time
start_time = time.time()

# declare seed sequence
#seed_string = "p47 p50 wait8 endp47 endp50 wait4 p47 p50 wait8 endp47 endp50"

# finally train the model
for epoch in range(50):
    
    # init the hidden and cell states to zero
    hc = net.init_hidden()
    
    # (x, y) refers to one batch with index i, where x is input, y is target
    for i, (x, y) in enumerate(get_batches(train_data, 128, 512)):
        
        # get the torch tensors from the one-hot of training data
        # also transpose the axis for the training set and the targets
        x_train = torch.from_numpy(to_categorical(x, num_classes=net.vocab_size).transpose([1, 0, 2]))
        targets = torch.from_numpy(y.T).type(torch.LongTensor)  # tensor of the target
        
        # if using gpu
        x_train = x_train.to(device)
        targets = targets.to(device)
        
        # zero out the gradients
        optimizer.zero_grad()
        
        # get the output sequence from the input and the initial hidden and cell states
        # calls forward function
        output = net(x_train, hc)
    
        # calculate the loss
        # we need to calculate the loss across all batches, so we have to flat the targets tensor
        loss = criterion(output, targets.contiguous().view(128*512))
        
        # calculate the gradients
        loss.backward()
        
        # update the parameters of the model
        optimizer.step()
        
        # track time
    
        # feedback every 100 batches
        if i % 100 == 0:
            
            # initialize the validation hidden state and cell state
            val_h, val_c = net.init_hidden()
            
            for val_x, val_y in get_batches(val_data, 128, 512):
        
                # prepare the validation inputs and targets
                val_x = torch.from_numpy(to_categorical(val_x).transpose([1, 0, 2]))
                val_y = torch.from_numpy(val_y.T).type(torch.LongTensor).contiguous().view(128*512)
  
                # if using gpu
                val_x = val_x.to(device)
                val_y = val_y.to(device)
            
                # get the validation output
                val_output = net(val_x, (val_h, val_c))
                
                # get the validation loss
                val_loss = criterion(val_output, val_y)
                
                # append the validation loss
                val_losses.append(val_loss.item())
                 
                # samples.append(''.join([int2char[int_] for int_ in net.predict("p33", seq_len=1024)]))
                
            #with open("training_output/loss/loss_epoch" + str(epoch) + "_batch" + str(i) + ".txt", "w") as loss_file:
            #    loss_file.write("Epoch: {}, Batch: {}, Train Loss: {:.6f}, Validation Loss: {:.6f}".format(epoch, i, loss.item(), val_loss.item()))

            with open("training_output/samples/beethoven_2/result_epoch" + str(epoch) + "_batch" + str(i) + ".txt", "w") as outfile:
                outfile.write(' '.join([int2word[int_] for int_ in net.predict('p33', pred_len=512)]))
        
            # track time
            duration = round(time.time() - start_time, 1)
            start_time = time.time()
    
            #print("Epoch: {}, Batch: {}, Duration: {} sec".format(epoch, i, duration))
            print("Epoch: {}, Batch: {}, Duration: {} sec, Train Loss: {:.6f}, Validation Loss: {:.6f}".format(epoch, i, duration, loss.item(), val_loss.item()))

    # save model every 5 epochs
    if epoch % 5 == 0:
        torch.save(net, 'models/beethoven_2/lstm' + str(epoch))
        
        
        

Epoch: 0, Batch: 0, Duration: 11.4 sec, Train Loss: 5.002234, Validation Loss: 4.978434
Epoch: 0, Batch: 100, Duration: 78.4 sec, Train Loss: 4.060242, Validation Loss: 4.235833
Epoch: 0, Batch: 200, Duration: 78.6 sec, Train Loss: 3.276564, Validation Loss: 3.489861


/jet/var/python/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type WordLSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 1, Batch: 0, Duration: 38.2 sec, Train Loss: 3.091501, Validation Loss: 3.317040
Epoch: 1, Batch: 100, Duration: 78.8 sec, Train Loss: 2.767900, Validation Loss: 2.905946
Epoch: 1, Batch: 200, Duration: 78.7 sec, Train Loss: 2.069426, Validation Loss: 2.284524
Epoch: 2, Batch: 0, Duration: 38.2 sec, Train Loss: 1.954763, Validation Loss: 2.176536
Epoch: 2, Batch: 100, Duration: 78.8 sec, Train Loss: 1.828510, Validation Loss: 1.958728
Epoch: 2, Batch: 200, Duration: 78.9 sec, Train Loss: 1.626389, Validation Loss: 1.825653
Epoch: 3, Batch: 0, Duration: 38.3 sec, Train Loss: 1.569928, Validation Loss: 1.784570
Epoch: 3, Batch: 100, Duration: 78.7 sec, Train Loss: 1.561190, Validation Loss: 1.715462
Epoch: 3, Batch: 200, Duration: 79.0 sec, Train Loss: 1.439928, Validation Loss: 1.659873
Epoch: 4, Batch: 0, Duration: 38.3 sec, Train Loss: 1.392178, Validation Loss: 1.638775
Epoch: 4, Batch: 100, Duration: 79.1 sec, Train Loss: 1.410460, Validation Loss: 1.612470
Epoch: 4, Batch: 2

Epoch: 31, Batch: 200, Duration: 79.1 sec, Train Loss: 0.512441, Validation Loss: 2.127385
Epoch: 32, Batch: 0, Duration: 38.3 sec, Train Loss: 0.481189, Validation Loss: 2.173853
Epoch: 32, Batch: 100, Duration: 79.3 sec, Train Loss: 0.507572, Validation Loss: 2.185253
Epoch: 32, Batch: 200, Duration: 78.3 sec, Train Loss: 0.500582, Validation Loss: 2.161828
Epoch: 33, Batch: 0, Duration: 37.6 sec, Train Loss: 0.468401, Validation Loss: 2.220862
Epoch: 33, Batch: 100, Duration: 79.0 sec, Train Loss: 0.507744, Validation Loss: 2.214358
Epoch: 33, Batch: 200, Duration: 79.4 sec, Train Loss: 0.494570, Validation Loss: 2.183249
Epoch: 34, Batch: 0, Duration: 38.5 sec, Train Loss: 0.462069, Validation Loss: 2.243927
Epoch: 34, Batch: 100, Duration: 79.5 sec, Train Loss: 0.492392, Validation Loss: 2.243418
Epoch: 34, Batch: 200, Duration: 79.2 sec, Train Loss: 0.482398, Validation Loss: 2.217270
Epoch: 35, Batch: 0, Duration: 38.5 sec, Train Loss: 0.452804, Validation Loss: 2.278739
Epoch: 

KeyboardInterrupt: 

In [ ]:
loaded_model = torch.load('../models/lstm' + str(epoch))